In [1]:
import os, re
import pandas as pd
import datetime
import nltk
import numpy as np

In [7]:
words_norust_nosw = pd.read_csv('../../Result/STM/alldf_norust_nosw_t45_w15_180302.csv')
words_norust_sw = pd.read_csv('../../Result/STM/alldf_norust_sw_t45_w15_180302.csv')
words_rust_nosw = pd.read_csv('../../Result/STM/alldf_rust_nosw_t45_w15_180302.csv')
words_rust_sw = pd.read_csv('../../Result/STM/alldf_rust_sw_t45_w15_180302.csv')
words_rust = pd.read_csv('../../Result/STM/alldf_rust_t45_w15_180302.csv')
words_norust = pd.read_csv('../../Result/STM/alldf_norust_t45_w15_180302.csv')

In [8]:
rust_sw = list(words_rust_sw.loc[33])[1:]
rust_nosw = list(words_rust_nosw.loc[12])[1:]
norust_sw = list(words_norust_sw.loc[35])[1:]
norust_nosw = list(words_norust_nosw.loc[4])[1:]
words_rust = list(words_rust.loc[43])[1:]
words_norust = list(words_norust.loc[12])[1:]

In [9]:
print(rust_sw)
print(rust_nosw)
print(norust_sw)
print(norust_nosw)
print(words_rust)
print(words_norust)

['trade', 'agreement', 'nafta', 'worker', 'manufactur', 'china', 'labor', 'deal', 'union', 'plant', 'green', 'steel', 'interest', 'special', 'oversea']
['ceo', 'subsidi', 'agreement', 'worker', 'export', 'trade', 'dollar', 'korea', 'crisi', 'propos', 'billion', 'farmer', 'south', 'offer', 'million']
['african-american', 'hillari', 'inner', 'clinton', 'islam', 'nafta', 'immigr', 'radic', 'massiv', 'trade', 'terror', 'includ', 'refuge', 'border', 'isi']
['latin', 'hemispher', 'cuba', 'cuban', 'religion', 'region', 'colombia', 'castro', 'trade', 'democraci', 'brazil', 'human', 'relationship', 'agreement', 'neighbor']
['trade', 'labor', 'worker', 'agreement', 'union', 'nafta', 'china', 'manufactur', 'play', 'plant', 'outsourc', 'wage', 'steel', 'organ', 'green']
['african-american', 'hillari', 'clinton', 'inner', 'unleash', 'email', 'nafta', 'disastr', 'donor', 'lie', 'trade', 'massiv', 'oppress', 'rig', 'fail']


In [10]:
words = list(set(rust_sw+rust_nosw+norust_sw+norust_nosw+words_rust+words_norust))

In [11]:
pt = nltk.stem.PorterStemmer()

In [12]:
rustbelts = ['New York', 'Pennsylvania', 'West Virginia', 'Ohio', 'Indiana', 'Michigan',
             'Illinois', 'Iowa', 'Wisconsin', 'Missouri']

In [13]:
rustbelts_ex = ['Pennsylvania', 'West Virginia', 'Ohio', 'Indiana', 'Michigan',
             'Illinois', 'Iowa', 'Wisconsin', 'Missouri']

In [14]:
dem = ['Barack Obama', 'Hillary Clinton', 'John Edwards', 'Bill Richardson',
      'Christopher Dodd', 'Joe Biden', 'Bernie Sanders',  "Martin O'Malley", 
      'Lincoln Chafee', 'Jim Webb']

rep = ['John McCain', 'Mike Huckabee', 'Mitt Romney', 'Rudy Giuliani',
      'Fred Thompson', 'Ron Paul', 'Newt Gingrich', 'Rick Santorum', 'Rick Perry',
      'Jon Huntsman', 'Michele Bachmann', 'Herman Cain', 'Tim Pawlenty',
       'Donald J. Trump', 'John Kasich', 'Ted Cruz', 'Marco Rubio', 'Ben Carson',
      'Jeb Bush', 'Chris Christie', 'Carly Fiorina', 'Rand Paul', 'George Pataki',
      'Lindsey Graham','Scott Walker', 'Bobby Jindal',]

In [15]:
stops = nltk.corpus.stopwords.words('english')

In [16]:
stemmed_stop = list(map(lambda x: re.sub('\W', '', x), stops))
punc = list(map(pt.stem , stemmed_stop))

In [17]:
rootDir = '../../Data/'
d = {}
remarkls = []
interviewls = []
questionls = []
otherls = []
speechls = []
addressls = []
r = 0
i = 0
q = 0
o = 0
s = 0
fdls = []
pe_dict = {2008:{}, 2012:{}, 2016:{}}
for dirName, subdirList, fileList in os.walk(rootDir) :
    print('Found directory: %s' % dirName)
    print(dirName)
    c = 0
    for fname in fileList:        
        if fname.endswith(".txt"):  
            filepath = os.path.join(dirName, fname)
            if dirName == '../../Data/PE2008':  
                c += 1
                year = 2008
                d = pe_dict[year]
                speaker = fname[8:-4]
            else:
                c += 1
                year = fname[8:12]
                d = pe_dict[int(year)]
                speaker = fname[12:-4]
            if speaker in dem:
                partyid = 'Dem'
            if speaker in rep:
                partyid = 'Rep'
            d[speaker]={}
            with open(filepath, 'r', -1, "ISO-8859-1") as input_file:
                data = input_file.read()
                splited_speeches = data.split('\n\n\n')
                #print(splited_speeches)
                statement_num = 0
                for i in range(len(splited_speeches)):
                    statement_num += 1
                    one_speech = re.sub(r'\((?!k\))(.*?)\)', '', splited_speeches[i], flags= re.IGNORECASE)
                    one_speech_fin = re.sub(r'\[(.*?)\]', '', one_speech) 
                    t = one_speech_fin.replace('\x80\x94','').replace('\x80','')\
                                      .replace('\x94','').split('\n')
                    text = [item for item in t if item != ''\
                            if not item.startswith('To view')\
                            if not item.startswith('Citation:')]
                    whole_text = " ".join(text[2:])    
                    #whole_text5 = re.sub(r'[^\w\s]', '',whole_text31.lower())                    
                    tokens = nltk.tokenize.word_tokenize(whole_text.lower())
                    stemmed_tokens = [pt.stem(w) for w in tokens]
                    #punc = [re.sub(r'[^\w]', '', word.lower()) for word in stops]
                    unigrams = [word for word in stemmed_tokens if word not in punc]                    
                    fd = nltk.FreqDist(unigrams)
                    if text != []:
                        title = text[0]
                        dateobj = datetime.datetime.strptime(text[1], '%B %d, %Y')
                        remarkls.append([speaker, title])
                        if 'remarks' in text[0].lower():
                            remarkls.append([speaker, title])
                            ttype = 'remarks'
                        elif 'speech' in text[0].lower():
                            speechls.append([speaker, title])
                            ttype = 'speech'
                        elif 'interview' in text[0].lower():
                            interviewls.append([speaker, title])
                            ttype = 'interview'
                        elif ('question' or 'q&a') in text[0].lower():
                            questionls.append([speaker, title])
                            ttype = 'question'
                        elif 'address' in text[0].lower():
                            addressls.append([speaker, title]) 
                            ttype = 'address'
                        else:
                            otherls.append([speaker, title])
                            ttype = 'others'
                                                
                        d[speaker][statement_num] = {'Author': speaker, 'Title': title,
                                                    'Year':year, 'Whole':splited_speeches[i],
                                                     'Clean': whole_text,
                                                     'Text': text,
                                                     'Date':dateobj,
                                                    'Type': ttype,
                                                     'PartyID':partyid,
                                                    'Length': len(unigrams)}
                        
                        fdls.append(fd.most_common(10))
                        for word in words:
                            num = unigrams.count(word)
                        #num = len(re.findall(word, speech.lower()))
                            d[speaker][statement_num].update({word: num})
                    #print(statement_num)

Found directory: ../../Data/
../../Data/
Found directory: ../../Data/cb_2016_us_state_20m
../../Data/cb_2016_us_state_20m
Found directory: ../../Data/PE2008
../../Data/PE2008
Found directory: ../../Data/PE2012
../../Data/PE2012
Found directory: ../../Data/PE2016
../../Data/PE2016


In [54]:
swing  = pd.read_excel('../Data/swingstate_2008-2016.xlsx')

In [18]:
temp = {}
for year1, values1 in pe_dict.items():
    for author1, values2 in values1.items():
        for number, values3 in values2.items():
            #print(values3)
            temp.setdefault('Year1', []).append(year1)
            temp.setdefault('Author1', []).append(author1)
            temp.setdefault('No.', []).append(number)
            for key, value in values3.items():
                temp.setdefault(key, []).append(value)

In [19]:
df = pd.DataFrame(temp)

In [65]:
#pd.read_csv('../../Data/pro_state_rust.csv')

In [66]:
#merge_dtm_fin[['State','pro_rust']].groupby('State').mean()

In [21]:
df08 = df[df['Year1'] == 2008]
df12 = df[df['Year1'] == 2012]
df16 = df[df['Year1'] == 2016]

In [22]:
f08 = df08[(datetime.datetime(2005, 1, 1, 0, 0) <= df08['Date']) \
           & (df08['Date'] <= datetime.datetime(2008, 12, 31, 0, 0))]

In [23]:
f12 = df12[(datetime.datetime(2009, 1, 1, 0, 0) <= df12['Date']) \
           & (df12['Date'] <= datetime.datetime(2012, 12, 31, 0, 0))]

In [24]:
f16 = df16[(datetime.datetime(2013, 1, 1, 0, 0) <= df16['Date']) \
           & (df16['Date'] <= datetime.datetime(2016, 12, 31, 0, 0))]

In [25]:
refine = pd.concat([f08,f12,f16]).reset_index()

In [26]:
refine.columns

Index(['index', 'Author', 'Author1', 'Clean', 'Date', 'Length', 'No.',
       'PartyID', 'Text', 'Title', 'Type', 'Whole', 'Year', 'Year1',
       'african-american', 'agreement', 'billion', 'border', 'brazil',
       'castro', 'ceo', 'china', 'clinton', 'colombia', 'crisi', 'cuba',
       'cuban', 'deal', 'democraci', 'disastr', 'dollar', 'donor', 'email',
       'export', 'fail', 'farmer', 'green', 'hemispher', 'hillari', 'human',
       'immigr', 'includ', 'inner', 'interest', 'isi', 'islam', 'korea',
       'labor', 'latin', 'lie', 'manufactur', 'massiv', 'million', 'nafta',
       'neighbor', 'offer', 'oppress', 'organ', 'outsourc', 'oversea', 'plant',
       'play', 'propos', 'radic', 'refuge', 'region', 'relationship',
       'religion', 'rig', 'south', 'special', 'steel', 'subsidi', 'terror',
       'trade', 'union', 'unleash', 'wage', 'worker'],
      dtype='object')

In [28]:
refine_fin  = refine[['Author', 'Author1', 'Clean', 'Date', 'Length', 'No.',
       'PartyID', 'Text', 'Title', 'Type', 'Whole', 'Year', 'Year1',
       'african-american', 'agreement', 'billion', 'border', 'brazil',
       'castro', 'ceo', 'china', 'clinton', 'colombia', 'crisi', 'cuba',
       'cuban', 'deal', 'democraci', 'disastr', 'dollar', 'donor', 'email',
       'export', 'fail', 'farmer', 'green', 'hemispher', 'hillari', 'human',
       'immigr', 'includ', 'inner', 'interest', 'isi', 'islam', 'korea',
       'labor', 'latin', 'lie', 'manufactur', 'massiv', 'million', 'nafta',
       'neighbor', 'offer', 'oppress', 'organ', 'outsourc', 'oversea', 'plant',
       'play', 'propos', 'radic', 'refuge', 'region', 'relationship',
       'religion', 'rig', 'south', 'special', 'steel', 'subsidi', 'terror',
       'trade', 'union', 'unleash', 'wage', 'worker']]

In [29]:
refine_fin.to_csv('../../Data/df_180306.csv')

In [30]:
rdf = pd.read_csv('../../Data/df_180306.csv')

In [31]:
rdf.columns

Index(['Unnamed: 0', 'Author', 'Author1', 'Clean', 'Date', 'Length', 'No.',
       'PartyID', 'Text', 'Title', 'Type', 'Whole', 'Year', 'Year1',
       'african-american', 'agreement', 'billion', 'border', 'brazil',
       'castro', 'ceo', 'china', 'clinton', 'colombia', 'crisi', 'cuba',
       'cuban', 'deal', 'democraci', 'disastr', 'dollar', 'donor', 'email',
       'export', 'fail', 'farmer', 'green', 'hemispher', 'hillari', 'human',
       'immigr', 'includ', 'inner', 'interest', 'isi', 'islam', 'korea',
       'labor', 'latin', 'lie', 'manufactur', 'massiv', 'million', 'nafta',
       'neighbor', 'offer', 'oppress', 'organ', 'outsourc', 'oversea', 'plant',
       'play', 'propos', 'radic', 'refuge', 'region', 'relationship',
       'religion', 'rig', 'south', 'special', 'steel', 'subsidi', 'terror',
       'trade', 'union', 'unleash', 'wage', 'worker'],
      dtype='object')

In [72]:
#pd.read_csv('../../Data/alldf_180223.csv').shape

In [32]:
covdf = pd.read_csv('../../Data/alldf_covariates2.csv')

In [33]:
covdf.columns

Index(['Unnamed: 0', 'Year', 'Year1', 'Author', 'Author1', 'Clean', 'Date',
       'No.', 'PartyID', 'State', 'Abb', 'Title', 'swing_last_08',
       'swing_last_12', 'swing_last_16', 'Rustbelt', 'unemployment',
       'challenger', 'governorp', 'primary'],
      dtype='object')

In [68]:
adddf = pd.read_excel('../../Data/df18022_addingstate_afternoon.xlsx', sheet_name='df18022_addingstate')

In [72]:
adddf.columns

Index(['index', 'Author', 'No.', 'Title', 'Type', 'Year', 'State', 'Abb'], dtype='object')

In [74]:
merged = refine_fin.merge(adddf, left_on=['Author','No.','Year'],
                right_on = ['Author','No.','Year'], how='left')

In [77]:
# load file contains state information
adddf = pd.read_excel('../../Data/df18022_addingstate_afternoon.xlsx', sheet_name='df18022_addingstate')

# merge and add state information
merged = refine_fin.merge(adddf, left_on=['Author','No.','Year'],
                right_on = ['Author','No.','Year'], how='left')

addingstatedf = merged[['Author', 'Author1', 'Clean', 'Date', 'Length', 'No.', 'PartyID',
       'Text', 'Title_x', 'Type_x', 'Whole', 'Year', 'Year1',
       'african-american', 'agreement', 'billion', 'border', 'brazil',
       'castro', 'ceo', 'china', 'clinton', 'colombia', 'crisi', 'cuba',
       'cuban', 'deal', 'democraci', 'disastr', 'dollar', 'donor', 'email',
       'export', 'fail', 'farmer', 'green', 'hemispher', 'hillari', 'human',
       'immigr', 'includ', 'inner', 'interest', 'isi', 'islam', 'korea',
       'labor', 'latin', 'lie', 'manufactur', 'massiv', 'million', 'nafta',
       'neighbor', 'offer', 'oppress', 'organ', 'outsourc', 'oversea', 'plant',
       'play', 'propos', 'radic', 'refuge', 'region', 'relationship',
       'religion', 'rig', 'south', 'special', 'steel', 'subsidi', 'terror',
       'trade', 'union', 'unleash', 'wage', 'worker', 'index', 'Title_y',
       'Type_y', 'State', 'Abb']]

addstate08 = addingstatedf[addingstatedf['Year1'] == 2008]
addstate12 = addingstatedf[addingstatedf['Year1'] == 2012]
addstate16 = addingstatedf[addingstatedf['Year1'] == 2016]

alldf = pd.concat([addstate08,addstate12,addstate16])

# load file contains swing state ratio information
swing  = pd.read_excel('../../Data/swingstate_2008-2016.xlsx')
sw08 = swing[swing['year'] == 2008]
sw12 = swing[swing['year'] == 2012]
sw16 = swing[swing['year'] == 2016]

# merge with original dataframe and add swing_last ration information
statesw08 = alldf.merge(sw08, left_on=['Abb'],right_on = ['state'], how='left')
statesw08.rename(columns = {'swing_last':'swing_last_08'}, inplace = True)
statesw12 = statesw08.merge(sw12, left_on=['Abb'],right_on = ['state'], how='left')
statesw12.rename(columns = {'swing_last':'swing_last_12'}, inplace = True)
statesw16 = statesw12.merge(sw16, left_on=['Abb'],right_on = ['state'], how='left')
statesw16.rename(columns = {'swing_last':'swing_last_16'}, inplace = True)

In [79]:
alldf_sw = statesw16[['Author', 'Author1', 'Clean', 'Date', 'State',
        'Length', 'No.', 'PartyID','Text', 'Title_x', 'Type_x', 
        'Whole', 'Year', 'Year1', 'african-american', 'agreement',
        'billion', 'border', 'brazil', 'castro', 'ceo', 'china', 
        'clinton', 'colombia', 'crisi', 'cuba',
        'cuban', 'deal', 'democraci', 'disastr', 'dollar', 'donor', 'email',
        'export', 'fail', 'farmer', 'green', 'hemispher', 'hillari', 'human',
        'immigr', 'includ', 'inner', 'interest', 'isi', 'islam', 'korea',
        'labor', 'latin', 'lie', 'manufactur', 'massiv', 'million', 'nafta',
        'neighbor', 'offer', 'oppress', 'organ', 'outsourc', 'oversea', 'plant',
        'play', 'propos', 'radic', 'refuge', 'region', 'relationship',
        'religion', 'rig', 'south', 'special', 'steel', 'subsidi', 'terror',
        'trade', 'union', 'unleash', 'wage', 'worker', 'index',
        'Abb', 'swing_last_08', 'swing_last_12', 'swing_last_16']]


In [80]:
alldf_sw ['Rustbelt'] = np.where(alldf_sw["State"].isin(rustbelts_ex), 1, 0)
alldf_sw.rename(columns = {'Title_x':'Title', 'Type_x':'Type'}, inplace = True)

/Users/misun/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/misun/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [83]:
covdf = pd.read_csv('../../Data/alldf_covariates2.csv')

In [86]:
alldf_sw.shape

(1247, 85)

In [84]:
merge_dtm = covdf.merge(alldf_sw, left_on=['Author','No.','Year','Year1',
                                      'Date','PartyID', 'Title'],
                             right_on = ['Author','No.','Year','Year1','Date',
                                      'PartyID', 'Title'],
                            how='left')

In [85]:
merge_dtm.columns

Index(['Unnamed: 0', 'Year', 'Year1', 'Author', 'Author1_x', 'Clean_x', 'Date',
       'No.', 'PartyID', 'State_x', 'Abb_x', 'Title', 'swing_last_08_x',
       'swing_last_12_x', 'swing_last_16_x', 'Rustbelt_x', 'unemployment',
       'challenger', 'governorp', 'primary', 'Author1_y', 'Clean_y', 'State_y',
       'Length', 'Text', 'Type', 'Whole', 'african-american', 'agreement',
       'billion', 'border', 'brazil', 'castro', 'ceo', 'china', 'clinton',
       'colombia', 'crisi', 'cuba', 'cuban', 'deal', 'democraci', 'disastr',
       'dollar', 'donor', 'email', 'export', 'fail', 'farmer', 'green',
       'hemispher', 'hillari', 'human', 'immigr', 'includ', 'inner',
       'interest', 'isi', 'islam', 'korea', 'labor', 'latin', 'lie',
       'manufactur', 'massiv', 'million', 'nafta', 'neighbor', 'offer',
       'oppress', 'organ', 'outsourc', 'oversea', 'plant', 'play', 'propos',
       'radic', 'refuge', 'region', 'relationship', 'religion', 'rig', 'south',
       'special', 'steel',

In [89]:
merge_dtm_fin = merge_dtm[['Unnamed: 0', 'index', 'Year', 'Year1', 'Author', 'Author1_x', 'Clean_x', 'Date',
       'No.', 'PartyID', 'State_x', 'Abb_x', 'Title', 'swing_last_08_x',
       'swing_last_12_x', 'swing_last_16_x', 'Rustbelt_x', 'unemployment',
       'challenger', 'governorp', 'primary',
       'Length', 'Text', 'Type', 'Whole', 'african-american', 'agreement',
       'billion', 'border', 'brazil', 'castro', 'ceo', 'china', 'clinton',
       'colombia', 'crisi', 'cuba', 'cuban', 'deal', 'democraci', 'disastr',
       'dollar', 'donor', 'email', 'export', 'fail', 'farmer', 'green',
       'hemispher', 'hillari', 'human', 'immigr', 'includ', 'inner',
       'interest', 'isi', 'islam', 'korea', 'labor', 'latin', 'lie',
       'manufactur', 'massiv', 'million', 'nafta', 'neighbor', 'offer',
       'oppress', 'organ', 'outsourc', 'oversea', 'plant', 'play', 'propos',
       'radic', 'refuge', 'region', 'relationship', 'religion', 'rig', 'south',
       'special', 'steel', 'subsidi', 'terror', 'trade', 'union', 'unleash',
       'wage', 'worker']]
merge_dtm_fin.rename(columns = {'Author1_x':'Author1','Clean_x':'Clean',
                                'State_x':'State','Abb_x':'Abb',
                                'swing_last_08_x':'swing_last_08',
                                'swing_last_12_x':'swing_last_12',
                                'swing_last_16_x':'swing_last_16',
                                'Rustbelt_x':'Rustbelt'}, inplace = True)

/Users/misun/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [90]:
merge_dtm_fin.columns

Index(['Unnamed: 0', 'index', 'Year', 'Year1', 'Author', 'Author1', 'Clean',
       'Date', 'No.', 'PartyID', 'State', 'Abb', 'Title', 'swing_last_08',
       'swing_last_12', 'swing_last_16', 'Rustbelt', 'unemployment',
       'challenger', 'governorp', 'primary', 'Length', 'Text', 'Type', 'Whole',
       'african-american', 'agreement', 'billion', 'border', 'brazil',
       'castro', 'ceo', 'china', 'clinton', 'colombia', 'crisi', 'cuba',
       'cuban', 'deal', 'democraci', 'disastr', 'dollar', 'donor', 'email',
       'export', 'fail', 'farmer', 'green', 'hemispher', 'hillari', 'human',
       'immigr', 'includ', 'inner', 'interest', 'isi', 'islam', 'korea',
       'labor', 'latin', 'lie', 'manufactur', 'massiv', 'million', 'nafta',
       'neighbor', 'offer', 'oppress', 'organ', 'outsourc', 'oversea', 'plant',
       'play', 'propos', 'radic', 'refuge', 'region', 'relationship',
       'religion', 'rig', 'south', 'special', 'steel', 'subsidi', 'terror',
       'trade', 'union', 'u

In [38]:
merge_dtm_fin['rust_sw_wcsum'] = merge_dtm_fin[rust_sw].sum(axis=1)
merge_dtm_fin['rust_nosw_wcsum'] = merge_dtm_fin[rust_nosw].sum(axis=1)
merge_dtm_fin['norust_sw_wcsum'] = merge_dtm_fin[norust_sw].sum(axis=1)
merge_dtm_fin['norust_nosw_wcsum'] = merge_dtm_fin[norust_nosw].sum(axis=1)
merge_dtm_fin['rust_wcsum'] = merge_dtm_fin[words_rust].sum(axis=1)
merge_dtm_fin['norust_wcsum'] = merge_dtm_fin[words_norust].sum(axis = 1)

/Users/misun/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/misun/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [39]:
merge_dtm_fin['pro_rust_sw'] = merge_dtm_fin['rust_sw_wcsum'] / merge_dtm_fin['Length']
merge_dtm_fin['pro_rust_nosw'] = merge_dtm_fin['rust_nosw_wcsum'] / merge_dtm_fin['Length']
merge_dtm_fin['pro_norust_sw'] = merge_dtm_fin['norust_sw_wcsum'] / merge_dtm_fin['Length']
merge_dtm_fin['pro_norust_nosw'] = merge_dtm_fin['norust_nosw_wcsum'] / merge_dtm_fin['Length']
merge_dtm_fin['pro_rust'] = merge_dtm_fin['rust_wcsum'] / merge_dtm_fin['Length']
merge_dtm_fin['pro_norust'] = merge_dtm_fin['norust_wcsum'] / merge_dtm_fin['Length']

In [46]:
merge_dtm_fin.to_csv('../../Data/alldf_w15_180306.csv')

In [50]:
merge_dtm_fin.groupby('State')['Year'].count().to_frame()

,Year
State,
alabama,3
arizona,6
arkansas,1
california,25
colorado,35
connecticut,4
dc,77
etc,285
florida,94


In [51]:
#merge_dtm_fin[merge_dtm_fin['State'] == 'utah']

In [74]:
#merged = rdf.merge(adddf, left_on=['Author','No.','Year'],
#                right_on = ['Author','No.','Year'], how='left')

In [75]:
#merged.columns

Index(['Unnamed: 0', 'Author', 'Author1', 'Clean', 'Date', 'No.', 'PartyID',
       'Text', 'Title_x', 'Type_x', 'Whole', 'Year', 'Year1', 'index',
       'Title_y', 'Type_y', 'State', 'Abb'],
      dtype='object')

In [44]:
#swing.columns

In [82]:
sw08 = swing[swing['year'] == 2008]
sw12 = swing[swing['year'] == 2012]
sw16 = swing[swing['year'] == 2016]

In [83]:
addingstatedf = merged[['Unnamed: 0', 'Author', 'Author1', 'Clean', 'Date', 'No.', 'PartyID',
                        'State','Abb','Text', 'Title_x', 'Type_x', 'Whole', 'Year', 'Year1', 
                        'index','Title_y', 'Type_y']]

In [87]:
addstate08 = addingstatedf[addingstatedf['Year1'] == 2008]
addstate12 = addingstatedf[addingstatedf['Year1'] == 2012]
addstate16 = addingstatedf[addingstatedf['Year1'] == 2016]

In [107]:
alldf = pd.concat([addstate08,addstate12,addstate16])

In [108]:
statesw08 = alldf.merge(sw08, left_on=['Abb'],right_on = ['state'], how='left')
statesw08.rename(columns = {'swing_last':'swing_last_08'}, inplace = True)

In [110]:
statesw12 = statesw08.merge(sw12, left_on=['Abb'],right_on = ['state'], how='left')
statesw12.rename(columns = {'swing_last':'swing_last_12'}, inplace = True)

In [112]:
statesw16 = statesw12.merge(sw16, left_on=['Abb'],right_on = ['state'], how='left')
statesw16.rename(columns = {'swing_last':'swing_last_16'}, inplace = True)

In [120]:
alldf_sw = statesw16[['Year', 'Year1', 'Author', 'Author1', 'Clean', 'Date', 'No.', 'PartyID',
       'State', 'Abb', 'Title_x', 'swing_last_08', 'swing_last_12', 'swing_last_16']]

In [123]:
alldf_sw ['Rustbelt'] = np.where(alldf_sw["State"].isin(rustbelts_ex), 1, 0)

/Users/misun/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [134]:
alldf_sw.rename(columns = {'Title_x':'Title'}, inplace = True)

/Users/misun/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [135]:
alldf_sw.columns

Index(['Year', 'Year1', 'Author', 'Author1', 'Clean', 'Date', 'No.', 'PartyID',
       'State', 'Abb', 'Title', 'swing_last_08', 'swing_last_12',
       'swing_last_16', 'Rustbelt'],
      dtype='object')

In [141]:
alldf_sw.to_csv('../Data/alldf_180226.csv')

In [140]:
#pd.read_csv('alldf_180223.csv')

In [104]:
print(addstate08.shape)
print(addstate12.shape)
print(addstate16.shape)
print(statesw08.shape)
print(statesw12.shape)
print(statesw16.shape)

(685, 18)
(282, 18)
(280, 18)
(685, 23)
(282, 23)
(280, 23)


In [44]:
rust10 = findf[findf['State'].isin(rustbelts_ex)][['Author','Year1','No.','Date',
                                                'PartyID','State','Clean']]

In [45]:
rust10_not = findf[~findf['State'].isin(rustbelts_ex)][['Author','Year1','No.','Date',
                                                        'PartyID','State','Clean']]

In [46]:
rust10_not.groupby('Year1').count()

,Author,No.,Date,PartyID,State,Clean
Year1,,,,,,
2008,510,510,510,510,510,510
2012,178,178,178,178,178,177
2016,206,206,206,206,206,206


In [121]:
#rust10_not.to_csv('non_rust_10_ex_180223.csv', index=False)

In [130]:
#pd.read_csv('non_rust_10_ex_180223.csv')

In [84]:
#rust10.to_csv('clean_rust_10_180223.csv')

In [85]:
#rust10_not.to_csv('clean_rust_10_not_180223.csv')

In [97]:
#clean = pd.read_csv('clean_rust_10_180223.csv')

In [99]:
#t = clean['Clean'].iloc[4]

In [118]:
#t

In [119]:
#clean[clean['Clean'].str.contains('[applause]')]

In [195]:
print(len(remarkls))
print(len(interviewls))
print(len(questionls))
print(len(otherls))
print(len(speechls))
print(len(addressls))

1069
178
2
17
53
79


In [42]:
#len(splited_speeches)

In [ ]:
temp = {}
for year1, values1 in pe_dict.items():
    for author1, values2 in values1.items():
        for number, values3 in values2.items():
            for num, values4 in values3.items():
                #print(values3)
                temp.setdefault('Doc.No.', []).append(number)
                temp.setdefault('Year1', []).append(year1)
                temp.setdefault('Author1', []).append(author1)
                temp.setdefault('No.', []).append(num)
                for key, value in values4.items():
                    temp.setdefault(key, []).append(value)